# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [23]:
# import the requests library (1 line)
import requests
# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"
# assign the /status endpoint to another variable called status_url (1 line)
status_url = f"{root_url}/status"
# query the /status endpoint using the get() method and store it in the req variable (1 line)
response = requests.get(status_url)
# check the status_code using a condition and print appropriate messages (4 lines)
if response.status_code == 200:
    print(response.text)
else:
    print(response.status_code)


"Alive"


### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [25]:
# Set the countries_url variable (1 line)
countries_url = f"{root_url}/countries"
# query the /countries endpoint using the get() method and store it in the req variable (1 line)
response = requests.get(countries_url)
# Get the JSON content and store it in the countries variable (1 line)
countries = response.json()
# display the request's status code and the countries variable (1 line)
print(response.status_code,countries)


403 {'message': 'The cookie is missing'}


### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [33]:
# Set the cookie_url variable (1 line)
cookie_url = f"{root_url}/cookie"

# Query the enpoint, set the cookies variable and display it (2 lines)
cookies = requests.get(cookie_url).json()
print(cookies)

{'message': 'The cookie has been created'}


Try to query the countries endpoint using the cookie, save the output and print it.

In [ ]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(countries_url).json()
# display the countries variable (1 line)
print(countries)

{'message': 'The cookie is missing'}


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [38]:
# Set the leaders_url variable (1 line)
leaders_url = f"{root_url}/leaders"
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url).json()
# display the leaders variable (1 line)
print(leaders)


{'message': 'The cookie is missing'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [108]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
requests.get(f"{leaders_url}", params=countries).json()
# assign the output to the leaders variable (1 line)
leaders = requests.get(f"{leaders_url}", params=countries).json()
# display the leaders variable (1 line)
print(leaders)

{'message': 'The cookie is missing'}


### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [ ]:
# 4 lines
def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    cookies = requests.get(f"{root_url}/cookie").cookies
    countries = requests.get(f"{root_url}/countries", cookies=cookies).json()
    leaders_per_country = {country: requests.get(f"{root_url}/leaders?country={country}", cookies=cookies).json() for country in countries}
    return leaders_per_country

print(get_leaders())


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# or 1 line
leaders_per_country[country] = requests.get(f"{root_url}/leaders?country={country}").json

# Print the dictionary (optional)
print(leaders_per_country)

{'message': '<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-not-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Genghis Khan/leaders - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [111]:
# < 15 lines
import requests
def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    cookies = requests.get(f"{root_url}/cookie").cookies
    countries = requests.get(f"{root_url}/countries", cookies=cookies).json()
    leaders_per_country = {}
    for country in countries:
        leaders_url = f"{root_url}/leaders?country={country}"
        response = requests.get(leaders_url, cookies=cookies)
        leaders_per_country[country] = response.json()
    return leaders_per_country
print(get_leaders())




{'ma': [{'id': 'Q57553', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1963-08-21', 'death_date': None, 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86', 'start_mandate': '1999-07-23', 'end_mandate': None}, {'id': 'Q69103', 'first_name': 'Hassan', 'last_name': 'None', 'birth_date': '1929-07-09', 'death_date': '1999-07-20', 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF', 'start_mandate': '1961-02-26', 'end_mandate': '1999-07-23'}, {'id': 'Q193874', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1909-08-10', 'death_date': '1961-02-26', 'place_of_birth': 'Fez', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [ ]:
# 2 lines






<function get_leaders at 0x000002515F34A160>


## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [122]:
# 3 lines
root_url = "http://en.wikipedia.org/wiki/Genghis_Khan"
query = requests.get(root_url).text
print(query)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="en" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Genghis Khan - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-featur

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [ ]:
# 3 lines
from bs4 import BeautifulSoup
soup = BeautifulSoup(query, "html.parser")
print(soup.prettify())


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Genghis Khan - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-en

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [ ]:
# 2 lines
for tag in soup.find_all("p"):
    print(tag.text)



Genghis Khan[a] (born Temüjin; c. 1162 – August 1227), also known as Chinggis Khan,[b] was the founder and first khan of the Mongol Empire. After spending most of his life uniting the Mongol tribes, he launched a series of military campaigns, conquering large parts of China and Central Asia.

Born between 1155 and 1167 and given the name Temüjin, he was the eldest child of Yesugei, a Mongol chieftain of the Borjigin clan, and his wife Hö'elün. When Temüjin was eight, his father died and his family was abandoned by its tribe. Reduced to near-poverty, Temüjin killed his older half-brother to secure his familial position. His charismatic personality helped to attract his first followers and to form alliances with two prominent steppe leaders named Jamukha and Toghrul; they worked together to retrieve Temüjin's newlywed wife Börte, who had been kidnapped by raiders. As his reputation grew, his relationship with Jamukha deteriorated into open warfare. Temüjin was badly defeated in c. 1187

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

In [129]:
# <10 lines
from bs4 import BeautifulSoup
root_url = "https://en.wikipedia.org/wiki/Khanate"
query = requests.get(root_url).text
soup = BeautifulSoup(query, "html.parser")
for tag in soup.find_all("p"):
    print(tag.text)




A khanate (/ˈxɑːneɪt, -ət/ KHAHN-ayt, -⁠ət) or khaganate refers to historic polities ruled by a khan, khagan, khatun, or khanum.[1][2] Khanates were typically nomadic Mongol and Turkic or Tatar societies located on the Eurasian Steppe,[3][4][5] and politically equivalent in status to kinship-based chiefdoms and feudal monarchies. Khanates and khaganates were organised tribally, where leaders gained power on the support and loyalty of their warrior subjects,[3] gaining tribute from subordinates as realm funding.[6] In comparison to a khanate, a khaganate, the realm of a khagan, was a large nomadic state maintaining subjugation over numerous smaller khanates.[7] The title of khagan, translating as "Khan of the Khans", roughly corresponds in status to that of an emperor.[4]

The Mongol Empire was the largest steppe nomadic Khaganate as well as second largest empire and the largest contiguous empire[8] in history. After Genghis Khan established appanages for his family in the Mongol Empir

At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [15]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph

In [16]:
# Test: 3 lines


### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [17]:
# 3 lines



Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [18]:
# 10 lines


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [19]:
# < 20 lines


## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [20]:
# < 20 lines

In [21]:
# Check the output of your function (2 lines)


Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [22]:
# < 25 lines



Check the output of your function again.

In [23]:
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [24]:
# < 20 lines


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [25]:
# <25 lines


Test your new functions.



## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [26]:
# 3 lines


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [27]:
# 3 lines


Make a function `save(leaders_per_country)` to call this code easily.

In [28]:
# 3 lines


In [29]:
# Call the function (1 line)


## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!